In [136]:
library(pracma)
library(matrixcalc)
library(fields)
library('latex2exp')
library(Matrix)
library(igraph)
#load source file
src.path <- "../../src"
source(paste(src.path, "DataGenerationProcess", "synth_basis.R", sep="/"))
source(paste(src.path, "DataGenerationProcess", "synth_data.R", sep="/"))
source(paste(src.path, "DataGenerationProcess", "synth_graph.R", sep="/"))
source(paste(src.path, "DataGenerationProcess", "synth_linearop.R", sep="/"))

source(paste(src.path, "Estimation", "basis_estimation.R", sep="/"))
source(paste(src.path, "Utility", "utility.R", sep="/"))
source(paste(src.path, "Estimation", "cca_estimation.R", sep="/"))
source(paste(src.path, "Utility", "R2python.R", sep="/"))

In [181]:
n <- 2
p <- 10
k.gen <- 11
M <- 3
obs.time <- seq(0,1,1/50)

cov_name <- 'random'

In [182]:
km.gen <- c(11,11,11)

Apinv_list <- list()
A_list <- list()
N_list <- list()
basis.m_list <- list()
true.basis_list <- list()
true.values_list <- list()



if (cov_name == "random"){
    omega <- synth.omega.random(p, k.gen, drop=0.3)
}
if (cov_name == "power"){
    omega <- synth.omega.power(p, k.gen)
    #omega <- solve(omega)
}
if (cov_name == "tridiag1"){
    omega <- synth.omega.tridiag1(p, k.gen)
}
if (cov_name == "tridiag2"){
    omega <- synth.omega.tridiag2(p, k.gen)
}
if (cov_name == "tridiag3"){
    omega <- synth.omega.tridiag3(p, k.gen)
}

#ensure that the diagonal values are all 1

G.true <- matrix(0, p, p) # p by p adjacency matrix
for(i in 1:p){
  for(j in 1:p){
    if(sum(abs(omega[((i-1)*k.gen+1):(i*k.gen), ((j-1)*k.gen+1):(j*k.gen)])) > 0)
      G.true[i,j] <- 1
  }
}

cov <- solve(omega)


for(m in 1:M){
    Am <- synth.linear_op.sparse_orthogonal(k.gen, km.gen[m], k.gen, scale=.2)
    Am <- t(t(Am) %*% diag(.2*((1:k.gen))+1)) #this is to make the singular values distinct
    #Am <- diag(.2*((1:k.gen)))
    A_list[[m]] <- Am
    Apinv_list[[m]] <- solve(Am)
}

for(m in 1:M){
    
    #noise covariance
    N_list[[m]] <- diag(p*km.gen[m])*0.5
    basis.m_list[[m]] <- synth.fourier.bases.m(obs.time, km.gen[m])
    if(m == 2){
         basis.m_list[[m]] <- synth.bspline.bases.m(obs.time, km.gen[m])
    }
}

#convert to regression B
B_list <- utility.graph2B(omega,p)
# save true graphs


#generate data 
set.seed(1)
data <- synth.data_from_graph(n, p, cov, basis.m_list, Apinv_list,N_list, dependent=TRUE, addnoise=FALSE)

for(m in 1:M){
    
    #noise covariance
    N_list[[m]] <- diag(p*km.gen[m])*0.0
}
set.seed(1)
noiseless_data <- synth.data_from_graph(n, p, cov, basis.m_list, Apinv_list,N_list, dependent=TRUE, addnoise=FALSE)

[1] "Generating data from modality 1"
[1] "Generating data from modality 2"
[1] "Generating data from modality 3"
[1] "Generating data from modality 1"
[1] "Generating data from modality 2"
[1] "Generating data from modality 3"


In [183]:
length(data )

[1] 3

In [208]:
#par(bg='white')
y = data[[1]][2,1,] + rnorm(51, 0,1)
min_y = min(noiseless_data[[1]][2,1,], data[[1]][2,1,])
max_y = max(noiseless_data[[1]][2,1,], data[[1]][2,1,])
tiff('M1_node1.tiff',  units="in", width=5, height=5, res=600)
plot(1, ylim=c(min_y-1,max_y+1), xlim=c(0,1), type="n", xlab='', ylab='', cex.lab=1.5, xaxt='n', yaxt='n')
lines(obs.time, noiseless_data[[1]][2,1,], col="black", lwd=2,  lty = "dashed")
lines(obs.time, data[[1]][2,1,], col="green", lwd=2)
title('Node 1', cex.main=2.5,line=0.5)#, ylab=TeX(r'($Y_{1,1}(t)$)'), cex.lab=2.5)
dev.off()

#par(bg='white')
min_y = min(noiseless_data[[2]][2,1,], data[[2]][2,1,])
max_y = max(noiseless_data[[2]][2,1,], data[[2]][2,1,])
tiff('M2_node1.tiff',  units="in", width=5, height=5, res=600)
plot(1, ylim=c(min_y-1,max_y+1), xlim=c(0,1), type="n", xlab='', ylab='', cex.lab=1.5, xaxt='n', yaxt='n')
lines(obs.time, noiseless_data[[2]][2,1,], col="black", lwd=2,  lty = "dashed")
lines(obs.time, data[[2]][2,1,], col="blue", lwd=2)
title('Node 1', cex.main=2.5,line=0.5)#, ylab=TeX(r'($Y_{2,1}(t)$)'), cex.lab=2.5)
dev.off()

#par(bg='white')
min_y = min(noiseless_data[[3]][2,1,], data[[3]][2,1,])
max_y = max(noiseless_data[[3]][2,1,], data[[3]][2,1,])
tiff('M3_node1.tiff',  units="in", width=5, height=5, res=600)
plot(1, ylim=c(min_y-1,max_y+1), xlim=c(0,1), type="n", xlab='', ylab='', cex.lab=1.5, xaxt='n', yaxt='n')
lines(obs.time, noiseless_data[[3]][2,1,], col="black", lwd=2,  lty = "dashed")
lines(obs.time, data[[3]][2,1,], col="red", lwd=2)
title('Node 1', cex.main=2.5,line=0.5)#, xlab='time', ylab=TeX(r'($Y_{M,1}(t)$)'), cex.lab=2.5)
dev.off()




#par(bg='white')
min_y = min(noiseless_data[[1]][2,2,], data[[1]][2,2,])
max_y = max(noiseless_data[[1]][2,2,], data[[1]][2,2,])
tiff('M1_node2.tiff',  units="in", width=5, height=5, res=600)
plot(1, ylim=c(min_y-1,max_y+1), xlim=c(0,1), type="n", xlab='', ylab='', cex.lab=1.5, xaxt='n', yaxt='n')
lines(obs.time, noiseless_data[[1]][2,2,], col="black", lwd=2,  lty = "dashed")
lines(obs.time, data[[1]][2,2,], col="green", lwd=2)
title('Node 2', cex.main=2.5,line=0.5)#, ylab=TeX(r'($Y_{1,2}(t)$)'), cex.lab=2.5)
dev.off()

#par(bg='white')
min_y = min(noiseless_data[[2]][2,2,], data[[2]][2,2,])
max_y = max(noiseless_data[[2]][2,2,], data[[2]][2,2,])
tiff('M2_node2.tiff',  units="in", width=5, height=5, res=600)
plot(1, ylim=c(min_y-1,max_y+1), xlim=c(0,1), type="n", xlab='', ylab='', cex.lab=1.5, xaxt='n', yaxt='n')
lines(obs.time, noiseless_data[[2]][2,2,], col="black", lwd=2,  lty = "dashed")
lines(obs.time, data[[2]][2,2,], col="blue", lwd=2)
title('Node 2', cex.main=2.5,line=0.5)#, ylab=TeX(r'($Y_{2,2}(t)$)'), cex.lab=2.5)
dev.off()

#par(bg='white')
min_y = min(noiseless_data[[3]][2,2,], data[[3]][2,2,])
max_y = max(noiseless_data[[3]][2,2,], data[[3]][2,2,])
tiff('M3_node2.tiff',  units="in", width=5, height=5, res=600)
plot(1, ylim=c(min_y-1,max_y+1), xlim=c(0,1), type="n", xlab='', ylab='', cex.lab=1.5, xaxt='n', yaxt='n')
lines(obs.time, noiseless_data[[3]][2,2,], col="black", lwd=2,  lty = "dashed")
lines(obs.time, data[[3]][2,2,], col="red", lwd=2)
title('Node 2', cex.main=2.5,line=0.5)#,xlab='time', ylab=TeX(r'($Y_{M,2}(t)$)'), cex.lab=2.5)
dev.off()





#par(bg='white')
min_y = min(noiseless_data[[1]][2,10,], data[[1]][2,10,])
max_y = max(noiseless_data[[1]][2,10,], data[[1]][2,10,])
tiff('M1_node10.tiff',  units="in", width=5, height=5, res=600)
plot(1, ylim=c(min_y-1,max_y+1), xlim=c(0,1), type="n", xlab='', ylab='', cex.lab=1.5, xaxt='n', yaxt='n')
lines(obs.time, noiseless_data[[1]][2,10,], col="black", lwd=2,  lty = "dashed")
lines(obs.time, data[[1]][2,10,], col="green", lwd=2)
title('Node 10', cex.main=2.5,line=0.5)#, ylab=TeX(r'($Y_{1,10}(t)$)'), cex.lab=2.5)
dev.off()

#par(bg='white')
min_y = min(noiseless_data[[2]][2,10,], data[[2]][2,10,])
max_y = max(noiseless_data[[2]][2,10,], data[[2]][2,10,])
tiff('M2_node10.tiff', units="in", width=5, height=5, res=600)
plot(1, ylim=c(min_y-1,max_y+1), xlim=c(0,1), type="n", xlab='', ylab='', cex.lab=1.5, xaxt='n', yaxt='n')
lines(obs.time, noiseless_data[[2]][2,10,], col="black", lwd=2,  lty = "dashed")
lines(obs.time, data[[2]][2,10,], col="blue", lwd=2)
title('Node 10', cex.main=2.5,line=0.5)#, ylab=TeX(r'($Y_{2,10}(t)$)'), cex.lab=2.5)
dev.off()

#par(bg='white')
min_y = min(noiseless_data[[3]][2,10,], data[[3]][2,10,])
max_y = max(noiseless_data[[3]][2,10,], data[[3]][2,10,])
tiff('M3_node10.tiff',  units="in", width=5, height=5, res=600)
plot(1, ylim=c(min_y-1,max_y+1), xlim=c(0,1), type="n", xlab='', ylab='', cex.lab=1.5, xaxt='n', yaxt='n')
lines(obs.time, noiseless_data[[3]][2,10,], col="black", lwd=2,  lty = "dashed")
lines(obs.time, data[[3]][2,10,], col="red", lwd=2)
title('Node 10', cex.main=2.5,line=0.5)#,xlab='time', ylab=TeX(r'($Y_{M,10}(t)$)'), cex.lab=2.5)
dev.off()

pdf 
  2

pdf 
  2

pdf 
  2

pdf 
  2

pdf 
  2

pdf 
  2

pdf 
  2

pdf 
  2

pdf 
  2

In [141]:
adj <- matrix(0,p,p)
for(i in 1:p){
    for(j in 1:p){
        if(i != j){
            xidx <- ((i-1)*k.gen+1):(i*k.gen)
            yidx <- ((j-1)*k.gen+1):(j*k.gen)
            subm <- norm(omega[xidx, yidx])
            if(subm>0) adj[i,j] <- 1
        }
    }
}

print(adj)

      [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
 [1,]    0    0    0    0    0    1    0    1    0     0
 [2,]    0    0    0    1    0    1    0    0    1     0
 [3,]    0    0    0    0    0    1    1    0    1     0
 [4,]    0    1    0    0    0    0    1    0    0     0
 [5,]    0    0    0    0    0    0    0    0    0     0
 [6,]    1    1    1    0    0    0    0    0    1     0
 [7,]    0    0    1    1    0    0    0    1    0     1
 [8,]    1    0    0    0    0    0    1    0    1     0
 [9,]    0    1    1    0    0    1    0    1    0     0
[10,]    0    0    0    0    0    0    1    0    0     0


In [207]:
network <- graph_from_adjacency_matrix(adj , mode='undirected', diag=F )
#par(bg='white')

tiff("latent_graph.tiff", units="in", width=5, height=5, res=600)
plot(network, layout=layout.circle)
title('Latent Graph', cex.main=2.5)

dev.off()


pdf 
  2